In [3]:
import numpy as np
import matplotlib.pyplot as plt
import qiskit as qk
import random

from generator import generateCliffordCircuit
# from execution.transpiler import transpileListToQiskitCircuit

In [4]:
def stringToQiskitSingleGate(gateString, qiskitCir, whichQubit):
    if gateString == 'I':
        qiskitCir.id(whichQubit)
    elif gateString == 'X':
        qiskitCir.x(whichQubit)
    elif gateString == 'Y':
        qiskitCir.y(whichQubit)
    elif gateString == 'Z':
        qiskitCir.z(whichQubit)
    elif gateString == 'H':
        qiskitCir.h(whichQubit)
    elif gateString == 'S':
        qiskitCir.s(whichQubit)
        
def transpileListToQiskitCircuit(cir):
    depth = len(cir)
    width = len(cir[0])
    qiskitCir = qk.QuantumCircuit(width)
    for d in range(width):
        if d % 2 == 0:
            for w in range(width):
                singleGate = cir[d][w]
                stringToQiskitSingleGate(singleGate, qiskitCir, w)
            if d != width - 1:
                qiskitCir.barrier()
        else:
            c = cir[d].index('CNOT_C')
            t = cir[d].index('CNOT_T')
            qiskitCir.cx(c, t)
            if d != width - 1:
                qiskitCir.barrier()
    return qiskitCir

In [95]:
width = 5 # num_qubits
depth = 5 # number of layers in the circuit (easy + hard)
singleGateSet = ['X', 'H', 'Z', 'I', 'S']
doubleGateSet = ['CNOT_C', 'CNOT_T']
twirlingGateSet = ['X','Y' 'Z', 'I']

circuit = generateCliffordCircuit(width, depth, singleGateSet, doubleGateSet)
qiskitCircuit = transpileListToQiskitCircuit(circuit)
print(circuit)
if depth < 10:
    print(qiskitCircuit)

[['S', 'I', 'I', 'S', 'X'], ['I', 'CNOT_C', 'I', 'CNOT_T', 'I'], ['H', 'H', 'H', 'H', 'Z'], ['CNOT_C', 'I', 'I', 'CNOT_T', 'I'], ['Z', 'I', 'H', 'S', 'I']]
     ┌───┐ ░       ░ ┌───┐ ░       ░ ┌───┐
q_0: ┤ S ├─░───────░─┤ H ├─░───■───░─┤ Z ├
     ├───┤ ░       ░ ├───┤ ░   │   ░ ├───┤
q_1: ┤ I ├─░───■───░─┤ H ├─░───┼───░─┤ I ├
     ├───┤ ░   │   ░ ├───┤ ░   │   ░ ├───┤
q_2: ┤ I ├─░───┼───░─┤ H ├─░───┼───░─┤ H ├
     ├───┤ ░ ┌─┴─┐ ░ ├───┤ ░ ┌─┴─┐ ░ ├───┤
q_3: ┤ S ├─░─┤ X ├─░─┤ H ├─░─┤ X ├─░─┤ S ├
     ├───┤ ░ └───┘ ░ ├───┤ ░ └───┘ ░ ├───┤
q_4: ┤ X ├─░───────░─┤ Z ├─░───────░─┤ I ├
     └───┘ ░       ░ └───┘ ░       ░ └───┘


In [96]:
def Clifford_Permute(cliff,pauli):
    '''This function permutes the pauli operators according to the clifford group, so it computes 
    P' = C P C^T, where C is a clifford operator and P is a pauli operator.
    Inputs: cliff - A Qiskit Circuit, Clifford, or Gate object.
            pauli - A Qiskit Pauli object.
    Outputs: new_pauli - A Qiskit Pauli object.'''

    return pauli.evolve(cliff,frame="s")


In [97]:
Clifford_Permute(qiskitCircuit, qk.quantum_info.Pauli('X'*width))

Pauli('XIXYY')

In [9]:
def split_circuit_by_barrier(circuit):
    qasm = circuit.qasm()
    prelude = []
    circuits = [[]]
    for line in qasm.splitlines():
        if any([line.startswith(t) for t in ['OPENQASM', 'include', 'qreg', 'creg']]):
            prelude.append(line)
        elif line.startswith('barrier'):
            circuits.append([])
        else:
            circuits[-1].append(line)
    circuits_with_prelude = [prelude+circuit for circuit in circuits]
    for circuit_with_prelude in circuits_with_prelude:
        yield qk.QuantumCircuit.from_qasm_str('\n'.join(circuit_with_prelude))

# for subcircuit in (split_circuit_by_barrier(qiskitCircuit)):
#     print(subcircuit)

In [76]:
def G_twirling(circuit):
    '''This function performs G-twirling on a circuit. It randomly chooses a Pauli from {X,Y,Z,I} and prepends it before a layer, then appends P'=CPC^T after the layer.}
    Inputs: circuit - A Qiskit Circuit object.
    Outputs: new_circuit - A Qiskit Circuit object.'''
    num_qubits = circuit.num_qubits
    new_circuit = qk.QuantumCircuit(num_qubits)
    for index, subcircuit in enumerate(split_circuit_by_barrier(circuit)):
        # if len(subcircuit.data) == 1:
        #     operated_qubits = [subcircuit.find_bit(q).index for q in subcircuit.data[0].qubits]
        # else:
        #     operated_qubits = list(range(num_qubits))
        pauli = qk.quantum_info.Pauli(''.join(random.choice(["X","Y","Z","I"]) for _ in range(num_qubits)))
        new_circuit = new_circuit.compose(pauli.to_instruction(),qubits=range(num_qubits))
        new_circuit = new_circuit.compose(subcircuit,qubits=range(num_qubits))
        new_circuit = new_circuit.compose(Clifford_Permute(subcircuit,pauli).to_instruction(),qubits = range(num_qubits))
        if index != len(list(split_circuit_by_barrier(circuit)))-1:
            new_circuit.barrier()
    return new_circuit


In [86]:
G_twirling(qiskitCircuit).draw()

┌───────────────┐┌───┐┌─────────┐ ░ ┌───────────────┐     »
q_0: ┤0              ├┤ H ├┤0        ├─░─┤0              ├─────»
     │               │├───┤│         │ ░ │               │     »
q_1: ┤1              ├┤ X ├┤1        ├─░─┤1              ├─────»
     │               │├───┤│         │ ░ │               │     »
q_2: ┤2 Pauli(YZZYY) ├┤ Z ├┤2 -YZZYY ├─░─┤2 Pauli(XYYXY) ├──■──»
     │               │├───┤│         │ ░ │               │  │  »
q_3: ┤3              ├┤ I ├┤3        ├─░─┤3              ├──┼──»
     │               │├───┤│         │ ░ │               │┌─┴─┐»
q_4: ┤4              ├┤ Z ├┤4        ├─░─┤4              ├┤ X ├»
     └───────────────┘└───┘└─────────┘ ░ └───────────────┘└───┘»
«     ┌───────────────┐ ░ ┌───────────────┐┌───┐┌─────────┐ ░ ┌───────────────┐»
«q_0: ┤0              ├─░─┤0              ├┤ I ├┤0        ├─░─┤0              ├»
«     │               │ ░ │               │├───┤│         │ ░ │               │»
«q_1: ┤1              ├─░─┤1              ├┤ X ├┤1        ├─░─┤1              ├»
«     │               │ ░ │               │├───┤│         │ ░ │               │»
«q_2: ┤2 Pauli(IYYXY) ├─░─┤2 Pauli(XZZYZ) ├┤ I ├┤2 -YXZYZ ├─░─┤2 Pauli(IYZZY) ├»
«     │               │ ░ │               │├───┤│         │ ░ │               │»
«q_3: ┤3              ├─░─┤3              ├┤ H ├┤3        ├─░─┤3              ├»
«     │               │ ░ │               │├───┤│         │ ░ │               │»
«q_4: ┤4              ├─░─┤4              ├┤ S ├┤4        ├─░─┤4              ├»
«     └───────────────┘ ░ └───────────────┘└───┘└─────────┘ ░ └───────────────┘»
«          ┌───────────────┐ ░ ┌───────────────┐┌───┐┌─────────┐
«q_0: ─────┤0              ├─░─┤0              ├┤ X ├┤0        ├
«          │               │ ░ │               │├───┤│         │
«q_1: ─────┤1              ├─░─┤1              ├┤ H ├┤1        ├
«     ┌───┐│               │ ░ │               │├───┤│         │
«q_2: ┤ X ├┤2 Pauli(IXYZY) ├─░─┤2 Pauli(XZXZY) ├┤ X ├┤2 -XZXXY ├
«     └─┬─┘│               │ ░ │               │├───┤│         │
«q_3: ──■──┤3              ├─░─┤3              ├┤ I ├┤3        ├
«          │               │ ░ │               │├───┤│         │
«q_4: ─────┤4              ├─░─┤4              ├┤ X ├┤4        ├
«          └───────────────┘ ░ └───────────────┘└───┘└─────────┘

In [84]:
circuit_ensemble = [G_twirling(qiskitCircuit) for _ in range(10)]
# for circ in circuit_ensemble:
#     print(circ)

In [ ]:
Lambda_est = 